<a href="https://colab.research.google.com/github/cesern/redes-neuronales/blob/master/redes_recurrentes/redes_recurrentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/cesern/redes-neuronales/blob/master/redes_recurrentes/imagenes/rn3.png?raw=1" width="200">
<img src="http://www.identidadbuho.uson.mx/assets/letragrama-rgb-150.jpg" width="200">

# [Curso de Redes Neuronales](https://curso-redes-neuronales-unison.github.io/Temario/)

# Redes recurrentes, implementación simple

[**Julio Waissman Vilanova**](http://mat.uson.mx/~juliowaissman/), 9 de mayo de 2018.

En esta libreta vamos a explorar como desarrollar redes recurrentes, simples y basadas en unidades de memoria de largo y corto plazo (LSTM), aplicadas a la generación automática de texto.

Dado que estamos en México en año electoral, y que se vota por diputados y presidentes municipales en muchisimos municipios del país, nos preguntamos si podríamos inventar nuevos municipios para que todos los candidatos tuvieran un lugar que gobernar. Así, generamos una lista con el nombre de todos los municipios de México, y la vamos a usar para aprender los nombres, y generar nombres a partir de una red recurrente. Esto es interesante ya que en México hay muchos municipios cuyos nombres tienen raices del español, el nahuatl, direrentes lenguas mayas, varias lenguas de la familia yuto-azteca, e inclusive algunos que son palabras inventadas (como Mexicali). Así que generar nombres de municipios mexicanos *creibles* es un problema interesante.

El archivo con el nombre de los municipios se encuentra para su descarga [aqui](municipios.txt).


## 1. Redes recurrentes: Desarrollar una red recurrente completamente a pie.

Con el fin de entender como funcionan las redes neuronales, vamos a aplicar un modelo de generación de texto *letra a letra*, en el cual tanto la arquitectura como el método de aprendizaje sean programados a mano. 

No vamos a pedir que lo programen todo solos, simplemente que utilicen el modelo programados en este [gist](https://gist.github.com/karpathy/d4dee566867f8291f086), y lo adapten a leer el nombre de los municipios y a generar nombres de municipios.

Para esto:

1. Copiiar el contenido del *gist* y comentarlo en español (y cambiar algo de código de forma que quede mñas claro para ti y para mi).

2. Copiar y comentar en español el contenido del método de verificción de gradiente (para limpiar el código de errores) y usarlo por unas cuantas iteraciones para demostrar que el algoritmo de entrenamiento funciona correctamente.

3. Ajustar los hiperparámetros del modelo, así como los parámetros del algoritmo de entrenamiento con el fin de generar una lista de nombres de municipios creibles, pero sin sobreaprendizaje (esto es, que copie vilmente el nombre de municipios existentes). 


In [0]:
import numpy as np

In [0]:
#Función de perdida
def loss_f(inputs, targets, hprev):
    """
    -imputs y targets arreglo de enteros
    -arreglo de Hx1 del estado inicial oculto
    -regresa el valor de perdida, gradientes de los parametros y el estado de la ultima capa oculta
     """
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0
    # forward pass
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
        ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
        loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
    # backward pass: compute gradients going backwards
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext # backprop into h
        dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(Whh.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [0]:
def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

In [5]:
#para usar los archivos de mi drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
!cp "/content/drive/My Drive/Colab Notebooks/Ultima/redes_recurrentes/municipios.txt" "municipios.txt" 
!ls

drive  municipios.txt  sample_data


In [7]:
# Lectura de datos
data = open('municipios.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print ('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# hiperparametros
hidden_size = 100 # tamaño de la capa oculta
seq_length = 25 # Número de pasos para desenrollar la RNN
learning_rate = 1e-1

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0

data has 35530 characters, 68 unique.


In [8]:
_max = 100000
while n < _max:
    # prepare inputs (we're sweeping from left to right in steps seq_length long)
    if p+seq_length+1 >= len(data) or n == 0: 
        hprev = np.zeros((hidden_size,1)) # reset RNN memory
        p = 0 # go from start of data
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
    
    #print('inputs: ', inputs)
    #print('targets: ', targets)
    
    # sample from the model now and then
    if n == _max - 1:
        sample_ix = sample(hprev, inputs[0], 200)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print ('----\n %s \n----' % (txt, ))

    # forward seq_length characters through the net and fetch gradient
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = loss_f(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % (_max/5) == 0: print ('iter %d, loss: %f' % (n, smooth_loss)) # print progress
  
    # perform parameter update with Adagrad
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

    p += seq_length # move data pointer
    n += 1 # iteration counter

iter 0, loss: 105.487698
iter 20000, loss: 41.616021
iter 40000, loss: 39.122717
iter 60000, loss: 38.468414
iter 80000, loss: 37.826173
----
 opolde
Jaltán de Mago Bea Villo
Mostol
La 
Jaquia
Momáus
Miutepec de Ló Ho
Novús
Marva
Matarca
Miguel Arto
Macha Malío Je C. Morrolapen de Lánare Toraco
Muguatla
Monela
Coatelacan
Lal Péngo Materde
Lu 
----


## Redes recurrentes tipo LSTM

Las redes con unidades LSTM las vimos platicadas en clase, pero no hay nada mejor para entender un tema que implementarlo y comparar los resultados con la red recurrente simple, sin memoria, para esto vamos a hacer lo mismo que antes, pero con unidades LSTM.

Para esto vamos a utilizar otro [gist](https://gist.github.com/karpathy/587454dc0146a6ae21fc) del mismo autor (que es una referencia obligada en el tema, por cierto). En este *gist*, el autor presenta un modelo de redes recurrentes LSTM desarrollado con *numpy* incluido el método de entrenamiento, pero no lo aplica a el modelado *letra a letra* como el gist pasado. Para esta parte de la libreta, lo que tienen que realizar es lo siguiente:


1. Copiiar el contenido del *gist* y comentarlo en español (y cambiar algo de código de forma que quede mñas claro para ti y para mi).

2. Adaptar el modelo propuesto para usarlo en la generación de nombres de municipios.

3. Ajustar los hiperparámetros del modelo, así como los parámetros del algoritmo de entrenamiento con el fin de generar una lista de nombres de municipios creibles, pero sin sobreaprendizaje (esto es, que copie vilmente el nombre de municipios existentes). 


In [0]:
# Agregar aqui el código, y en cuantas celdas como consideres necesarias




Por último, agrega en esta misma celda comentarios sobre la comparación entre los dos modelos vistos, diferencias, similitudes, ventajas, desventajas. Recuerda es una opinión personal basada en tu trabajo, no pongas lo que dice la literatura si no lo que tu experimentaste a la hora de desarrollar y aplicar los modelos, así no concuerde con lo que leas en otros lados.